In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import keras.ops as K
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.metrics import BinaryAccuracy

# from keras.models import Sequential
from deel.lip.model import Sequential

from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.activations import GroupSort, GroupSort2
from deel.lip.losses import HKR, KR, HingeMargin, MulticlassHKR, MulticlassKR

import numpy as np
import decomon

from data_processing import load_data, select_data_for_radius_evaluation_MNIST08
from radius_evaluation_tools import compute_binary_certificate, starting_point_dichotomy

import matplotlib.pyplot as plt

In [2]:
x_train, x_test, y_train, y_test, y_test_ord = load_data("MNIST08")

In [3]:
model_path = "/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras"
model = keras.models.load_model(model_path)
model.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
model_bis = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_4logits.keras")
model_bis.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
images, labels, idx_list = select_data_for_radius_evaluation_MNIST08(x_test, y_test_ord, model_bis)

2.5725006639080936

In [6]:
from lipschitz_decomon_tools import get_local_maximum, echantillonner_boule_l2_simple, square_backward_bounds, function_to_optimize_all


In [9]:
pt_choosen = 54
x = images[pt_choosen:pt_choosen+1].flatten().detach().cpu().numpy()
label = labels[pt_choosen:pt_choosen+1]
eps=1
nb_pts = 10

In [52]:
y_list = []
for i in range(nb_pts):
    ech = echantillonner_boule_l2_simple(x, eps)
    print(np.linalg.norm(x-ech))
    y_list.append(ech)

0.9988040007996434
0.9990695194835147
0.998553256194848
0.9971594947091237
0.999892190148236
0.9999036782346238
0.9993460787072705
0.999790039737079
0.9997715260729734
0.9987416697496079


In [23]:
x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)

In [25]:
f_adv

23.899818013380447

In [26]:
from radius_evaluation_tools import single_compute_relaxation_radius

In [13]:
# eps_working = single_compute_relaxation_radius(pt_choosen, images, labels, model, nb_pts)

In [14]:
# eps_working

In [ ]:
# y_list = [x]
# f_list = [get_local_maximum(x, label, eps, y_list, model)[1]]
# for i in range(10):
#     print(i)
#     i = 2*i
#     nb_pts = i+1
#     y_list.append(echantillonner_boule_l2_simple(x, eps))
#     x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)
#     f_list.append(f_adv)

# list_i = []
# for i in range(3):
#     list_i.append(2*i)


# plt.scatter(list_i, f_list)
# plt.xlabel("nb yi")
# plt.ylabel("max function")
# plt.show()

0
1
2
3
4
5
6
7
8
9


In [91]:
x_ech = echantillonner_boule_l2_simple(x, eps)
print(np.linalg.norm(x-x_ech))
x_ball_center = np.asarray(x, dtype=np.float64)

l = x-eps
u = x+eps

W_list = []
b_list = []
for y_i in y_list:
    # W, b = square_backward_bounds(l,u,y_i)
    W, b = square_backward_bounds(l,u,y_i)
    print(np.sum(W!=0), b)
    W_list.append(W)
    b_list.append(b)
    
function_to_optimize_all(x_ech, label, W_list, b_list, y_list, model)

0.999907658916056
784 [786.03916835]
784 [785.33045112]
784 [785.21703527]
784 [785.01718725]
784 [785.60688476]
784 [783.84708906]
784 [783.4960444]
784 [785.4087855]
784 [785.65487362]
784 [784.01400884]


23.877647887644482

In [32]:
np.sum(y_list[5] - y_list[1] != 0)

784

In [93]:
y_list = []
y_list.append(x)

for i in range(20):
    x_adv, f_adv = get_local_maximum(x, label, eps, y_list, model)
    y_list.append(x_adv)

In [94]:
f_adv

135.89739990234375

In [215]:
np.sum(W_list[5] - W_list[1] != 0)

0

In [100]:
import pdb

In [33]:
eps = 1

In [40]:
def square_backward_bounds_reduced(l, u, y):
    # l (4,)
    # u (4,)
    # y (4,)
    l = l[151:153]
    u = u[151:153]
    y = y[151:153]
    print(l, u, y)
    u = u - y
    l = l - y
    print(u, l)

    W = u + l #(4,)
    print(W)
    b = np.sum(-u*l) - W@y #scalar
    # pdb.set_trace()
    return W, np.array(b)[None]#(4,) & (1,)

In [35]:
v = x-eps
idx = np.nonzero(x)

In [36]:
idx

(array([151, 152, 153, 154, 155, 156, 179, 180, 181, 182, 183, 184, 185,
        206, 207, 208, 209, 210, 211, 212, 213, 214, 234, 235, 236, 237,
        239, 240, 241, 242, 261, 262, 263, 264, 265, 268, 269, 270, 289,
        290, 291, 292, 296, 297, 298, 317, 318, 319, 323, 324, 325, 345,
        346, 347, 348, 350, 351, 352, 353, 373, 374, 375, 376, 377, 378,
        379, 380, 402, 403, 404, 405, 406, 407, 408, 431, 432, 433, 434,
        435, 436, 459, 460, 461, 462, 463, 464, 487, 488, 489, 490, 491,
        492, 514, 515, 516, 517, 519, 520, 521, 542, 543, 544, 545, 547,
        548, 549, 570, 571, 572, 575, 576, 577, 598, 599, 600, 601, 602,
        603, 604, 605, 626, 627, 628, 629, 630, 631, 632, 633, 655, 656,
        657, 658, 659, 660, 661, 685, 686, 687, 688]),)

In [53]:
y_list[5][151:153]

array([-0.00076421,  0.66070286])

In [55]:
x[151:153]

array([0.01960784, 0.627451  ], dtype=float32)

In [56]:
W1, b1 = square_backward_bounds_reduced(x-eps, x+eps, y_list[2])

[-0.98039216 -0.372549  ] [1.0196079 1.627451 ] [0.021841   0.66357108]
[0.99776691 0.96387987] [-1.00223315 -1.03612007]
[-0.00446625 -0.07224021]


In [ ]:
W3, b3 = square_backward_bounds_reduced(np.asarray(x, dtype=np.float32)-eps, np.asarray(x, dtype=np.float32)+eps, y_list[2])

In [60]:
W2, b2 = square_backward_bounds_reduced(np.asarray(x, dtype=np.float64)-eps, np.asarray(x, dtype=np.float64)+eps, y_list[5])

[-0.98039216 -0.372549  ] [1.01960784 1.627451  ] [-0.00076421  0.66070286]
[1.02037205 0.96674815] [-0.97962795 -1.03325185]
[ 0.0407441  -0.06650371]


In [58]:
W1

array([-0.00446625, -0.07224021])

In [120]:
W1

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [61]:
x_ech = echantillonner_boule_l2_simple(x, eps)

In [64]:
function_to_optimize_all(x_ech, label, [W1, W2], [b1, b2], y_list[151:153], model)

UnboundLocalError: local variable 'function' referenced before assignment

In [68]:
outputs = []
for i in range(len(y_list)):
    output = model(y_list[i].reshape((1,28,28))[None]).cpu().detach().numpy()[0,0] +\
                1*np.sqrt(W_list[i]@x+b_list[i]) #scalar
    outputs.append(output)
function = np.min(outputs)

In [69]:
function

23.876823497231385

In [71]:
model(images[pt_choosen:pt_choosen+1])

tensor([[-4.1026]], device='cuda:0', grad_fn=<MmBackward0>)